In [1]:
import os

if "run_molecular_property_based_filtering.py" not in os.listdir():
    os.chdir("../src")

os.listdir()

['database',
 'utils',
 'screening',
 'run_molecular_property_based_filtering.py',
 '__init__.py']

---

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(
    "../data/chembl_34_initial_screening_results.csv", sep=",", index_col=0
)

In [4]:
display(df.head())
df.shape

,Lipinski_Rule_of_5,Ghose_Filter,Veber_Filter,Rule_of_3_Filter,REOS_Filter,Drug_like_Filter,QED_Filter,TPSA_Filter,SA_Score_Filter,NP_Score_Filter,Errors
ChemblID,,,,,,,,,,,
CHEMBL153534,True,False,True,False,True,True,0.608781,82.22,3.048474,-1.552303,[]
CHEMBL440060,False,False,False,False,False,False,0.016357,1271.93,9.084590,-0.037746,[]
CHEMBL440245,False,False,False,False,False,False,0.010798,1502.95,9.752879,0.219818,[]
CHEMBL440249,False,False,False,False,False,False,0.016726,979.36,9.146251,0.632360,[]
CHEMBL405398,False,True,True,False,True,False,0.623114,75.20,2.595185,-1.628093,[]


(2409257, 11)

(334969, 11)


In [30]:
def get_filter_molecule(
    QED_Filter_thereshold: float,
    SA_Score_Filter_threshold: float,
    NP_Score_Filter_threshold: float,
    ADMET_Scores_Filter: float,
) -> callable:
    def filter_molecule(row):
        return (
            # row["Lipinski_Rule_of_5"]
            # and
            row["Ghose_Filter"]
            and row["REOS_Filter"]
            and row["Veber_Filter"]
            and row["Drug_like_Filter"]
            and row["QED_Filter"] >= QED_Filter_thereshold
            and row["SA_Score_Filter"] <= SA_Score_Filter_threshold
            and row["NP_Score_Filter"] >= NP_Score_Filter_threshold
            # and row["ADMET_Scores_Filter"] <= ADMET_Scores_Filter
        )

    return filter_molecule

In [31]:
filter_fn = get_filter_molecule(
    QED_Filter_thereshold=0.5,  # 0.7,
    SA_Score_Filter_threshold=4,  # 3,
    NP_Score_Filter_threshold=0.2,
    ADMET_Scores_Filter=2.5,
)

---

In [32]:
df_admet = pd.read_csv("../data/admet.csv", sep=",", index_col=0)

In [33]:
df_admet

,ADMET_Scores_Filter,Errors
ChemblID,,
CHEMBL153534,1.138080,[]
CHEMBL440060,2570.754431,[]
CHEMBL440245,3230.738108,[]
CHEMBL440249,2274.126532,[]
CHEMBL405398,98.871021,[]
...,...,...
CHEMBL4298695,443.825858,[]
CHEMBL4298698,227.671867,[]
CHEMBL4298702,150.560680,[]


In [ ]:
df_corrected_lipinski5_and_druglike = pd.read_csv(
    "../data/corrected_lipinski5_and_druglikeness.csv", sep=",", index_col=0
)

---

In [ ]:
df_combined = (
    df[
        [
            "Ghose_Filter",
            "Veber_Filter",
            "REOS_Filter",
            "QED_Filter",
            "SA_Score_Filter",
            "NP_Score_Filter",
        ]
    ]
    .join(df_admet[["ADMET_Scores_Filter"]], lsuffix="_df", rsuffix="_admet")
    .join(
        df_corrected_lipinski5_and_druglike[["Lipinski_Rule_of_5", "Drug_like_Filter"]],
        lsuffix="_df",
        rsuffix="_lipinski_druglike",
    )
)
df_combined

,Lipinski_Rule_of_5,Ghose_Filter,Veber_Filter,Rule_of_3_Filter,REOS_Filter,Drug_like_Filter,QED_Filter,TPSA_Filter,SA_Score_Filter,NP_Score_Filter,Errors_df,ADMET_Scores_Filter,Errors_admet
ChemblID,,,,,,,,,,,,,
CHEMBL153534,True,False,True,False,True,True,0.608781,82.22,3.048474,-1.552303,[],1.138080,[]
CHEMBL440060,False,False,False,False,False,False,0.016357,1271.93,9.084590,-0.037746,[],2570.754431,[]
CHEMBL440245,False,False,False,False,False,False,0.010798,1502.95,9.752879,0.219818,[],3230.738108,[]
CHEMBL440249,False,False,False,False,False,False,0.016726,979.36,9.146251,0.632360,[],2274.126532,[]
CHEMBL405398,False,True,True,False,True,False,0.623114,75.20,2.595185,-1.628093,[],98.871021,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
CHEMBL4297438,False,True,True,False,False,False,0.432167,107.30,2.609728,-0.750485,[],77.401459,[]
CHEMBL4298636,False,False,False,False,False,False,0.040094,530.87,7.525295,0.447542,[],1095.926541,[]
CHEMBL4296948,True,False,True,False,True,True,0.484395,54.90,3.201334,-1.142129,[],0.671320,[]


In [35]:
filtered_df = df_combined[df_combined.apply(filter_fn, axis=1)]

In [36]:
filtered_df

,Lipinski_Rule_of_5,Ghose_Filter,Veber_Filter,Rule_of_3_Filter,REOS_Filter,Drug_like_Filter,QED_Filter,TPSA_Filter,SA_Score_Filter,NP_Score_Filter,Errors_df,ADMET_Scores_Filter,Errors_admet
ChemblID,,,,,,,,,,,,,
CHEMBL503634,False,True,True,False,True,True,0.553150,114.00,2.658629,0.996593,[],0.371830,[]
CHEMBL444368,True,True,True,True,True,True,0.844289,43.70,3.029199,1.461576,[],0.136600,[]
CHEMBL444434,False,True,True,False,True,True,0.609140,92.04,3.479000,2.172817,[],16.105353,[]
CHEMBL503603,True,True,True,False,True,True,0.615338,40.80,2.946122,0.985238,[],57.117686,[]
CHEMBL503469,True,True,True,False,True,True,0.539093,84.47,2.653626,0.207118,[],28.792657,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
CHEMBL4297312,True,True,True,False,True,True,0.839482,25.61,3.610542,0.912575,[],1.278500,[]
CHEMBL4296991,True,True,True,False,True,True,0.758058,74.60,2.918915,1.059271,[],21.142631,[]
CHEMBL4296988,True,True,True,False,True,True,0.772957,65.56,3.874596,1.167360,[],24.341443,[]


In [13]:
inh = pd.read_csv("../data/SMILES_inh.csv", sep=",", index_col=0)

In [14]:
from rdkit import Chem

known_inhibitors_mol = {}

for row in inh.iterrows():
    mol = Chem.MolFromSmiles(row[1]["smiles"])

    known_inhibitors_mol[row[0]] = mol

In [22]:
# for row in inh.iterrows():
#     mol = Chem.MolFromSmiles(row[1]["smiles"])
#     print(
#         f"molecular weight: {Chem.Descriptors.MolWt(mol)} | logP: {Chem.Descriptors.MolLogP(mol)} | HBA: {Chem.Descriptors.NumHAcceptors(mol)} | HBD: {Chem.Descriptors.NumHDonors(mol)}"
#     )
#     display(mol)

In [ ]:
from screening import MolecularPropertyCalculator

In [ ]:
mpc = MolecularPropertyCalculator()

In [ ]:
len(known_inhibitors_mol)

19

In [ ]:
results_df = pd.DataFrame(columns=["inhibitor_id", *mpc.get_result_keys()])

In [ ]:
for key, value in known_inhibitors_mol.items():
    basic_filters = mpc.calculate_basic_filters(value)
    results_df = pd.concat(
        [results_df, pd.DataFrame([{**{"inhibitor_id": key}, **basic_filters}])],
        ignore_index=True,
    )

results_df

reading NP model ...


model in
[22:15:57] DEPRECATION WARNING: please use MorganGenerator
/tmp/ipykernel_334073/1966216618.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat(
reading NP model ...
model in
[22:15:57] DEPRECATION WARNING: please use MorganGenerator
reading NP model ...
model in
[22:15:58] DEPRECATION WARNING: please use MorganGenerator
reading NP model ...
model in
[22:15:58] DEPRECATION WARNING: please use MorganGenerator
reading NP model ...
model in
[22:15:58] DEPRECATION WARNING: please use MorganGenerator
reading NP model ...
model in
[22:15:58] DEPRECATION WARNING: please use MorganGenerator
reading NP model ...
model in
[22:15:59] DEPRECATION WARNING: please use MorganGenerator
reading NP model ...
model in
[22:15:59

,inhibitor_id,Lipinski_Rule_of_5,Ghose_Filter,Veber_Filter,Rule_of_3_Filter,REOS_Filter,Drug_like_Filter,QED_Filter,TPSA_Filter,SA_Score_Filter,NP_Score_Filter,ADMET_Scores_Filter,Errors
0,0,False,False,True,False,False,False,0.374388,136.61,3.339819,-1.079381,181.359743,[]
1,1,False,False,True,False,True,False,0.538384,84.57,3.051925,-1.181657,151.249145,[]
2,2,False,False,True,False,True,False,0.516054,103.03,3.147472,-1.294599,155.478274,[]
3,3,False,True,True,False,True,False,0.670319,78.09,2.467244,-1.774178,40.402747,[]
4,4,False,True,True,False,True,False,0.599323,78.09,2.589870,-1.701862,95.762497,[]
5,5,False,True,True,False,True,False,0.442149,78.09,2.422425,-1.536256,103.598697,[]
6,6,False,True,True,False,True,False,0.441459,86.88,2.320230,-1.300281,74.835097,[]
7,7,False,True,False,False,False,False,0.375531,86.88,2.464274,-1.080771,98.073647,[]
8,8,False,True,True,False,True,False,0.596398,86.88,2.460566,-1.216697,66.998897,[]
9,9,False,False,False,False,False,False,0.258308,78.09,2.512693,-1.285886,161.221697,[]


In [ ]:
# co na czym przeszło i co nie + smiles

In [ ]:
results_df.apply(filter_fn, axis=1)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
dtype: bool

In [ ]:
filtered_resuts_df = results_df[results_df.apply(filter_fn, axis=1)]

In [ ]:
filtered_resuts_df

,inhibitor_id,Lipinski_Rule_of_5,Ghose_Filter,Veber_Filter,Rule_of_3_Filter,REOS_Filter,Drug_like_Filter,QED_Filter,TPSA_Filter,SA_Score_Filter,NP_Score_Filter,ADMET_Scores_Filter,Errors


---

In [41]:
import requests


def get_smiles_from_chembl(chembl_id):
    url = f"https://www.ebi.ac.uk/chembl/api/data/molecule/{chembl_id}"
    response = requests.get(url)
    if response.status_code == 200:
        try:
            data = response.json()
            return data.get("molecule_structures", {}).get("canonical_smiles", None)
        except requests.JSONDecodeError:
            print(f"Failed to decode JSON for {chembl_id}")
            return None
    else:
        print(
            f"Failed to retrieve data for {chembl_id}, status code: {response.status_code}"
        )
        return None


# Example usage
chembl_id = "CHEMBL503634"
smiles = get_smiles_from_chembl(chembl_id)
if smiles:
    print(f"{chembl_id}: {smiles}")
else:
    print(f"SMILES not found for {chembl_id}")

Failed to decode JSON for CHEMBL503634
SMILES not found for CHEMBL503634


In [40]:
get_smiles_from_chembl("CHEMBL503634")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)